# Amplitude embedding (quantum hardware)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on real quantum hardware --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [2]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [11]:
#image of the sky filled with double precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[6.0586487e+01+9.7869774e+01j 6.0586487e+01+9.7869774e+01j
  3.3984894e-03+2.9080615e-03j 6.0586487e+01+9.7869774e+01j]
 [6.0586487e+01+9.7869774e+01j 4.2823921e-03+5.6767180e-03j
  3.0596580e-03+7.4592889e-03j 5.5594412e-03+7.5533462e-04j]
 [7.9118107e-03+8.4970510e-03j 7.3066009e-03+1.4608257e-04j
  9.8789205e-05+5.6169359e-03j 6.2378724e-03+7.7958778e-03j]
 [6.0586487e+01+9.7869774e+01j 6.4992905e-03+6.9446545e-03j
  9.1194632e-03+3.4152376e-03j 6.8530305e-03+5.4909494e-03j]]


## 2) Applying a 2D FT (Fourier domain)

In [12]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 302.99277 +489.40356j   181.75043 +293.59897j    60.573338 +97.87086j
   181.75298 +293.59787j ]
 [ 181.73935 +293.59973j    60.579803 +97.85553j   -60.56751  -97.87049j
    60.562656 +97.86032j ]
 [  60.57607  +97.865j     -60.58392  -97.861626j -181.75058 -293.59225j
   -60.55587  -97.856224j]
 [ 181.74327 +293.5806j     60.586044 +97.87458j   -60.5876   -97.87559j
    60.572594 +97.86551j ]]


#### Sanity check

In [13]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[6.0586487e+01+9.7869774e+01j 6.0586491e+01+9.7869774e+01j
  3.3996105e-03+2.9053688e-03j 6.0586487e+01+9.7869774e+01j]
 [6.0586487e+01+9.7869774e+01j 4.2841434e-03+5.6753159e-03j
  3.0596256e-03+7.4586868e-03j 5.5587292e-03+7.5483322e-04j]
 [7.9123974e-03+8.4977150e-03j 7.3082447e-03+1.4734268e-04j
  9.7990036e-05+5.6166649e-03j 6.2353611e-03+7.7967644e-03j]
 [6.0586487e+01+9.7869774e+01j 6.5009594e-03+6.9441795e-03j
  9.1183186e-03+3.4151077e-03j 6.8514347e-03+5.4893494e-03j]]

Reals MSE: 2.251104e-12
Imaginaries MSE: 9.700789e-13


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [14]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.29430127+0.4753648j   0.17653683+0.28517696j  0.05883576+0.09506339j
  0.17653932+0.28517589j  0.17652607+0.28517768j  0.05884204+0.0950485j
 -0.0588301 -0.09506303j  0.05882539+0.09505315j  0.05883842+0.0950577j
 -0.05884604-0.09505442j -0.17653698-0.28517044j -0.0588188 -0.09504917j
  0.17652988+0.28515911j  0.05884811+0.095067j   -0.05884962-0.09506799j
  0.05883504+0.0950582j ]

[[ 302.99274 +489.40356j   181.75041 +293.59897j    60.573338 +97.87086j
   181.75298 +293.59787j ]
 [ 181.73935 +293.5997j     60.579803 +97.85553j   -60.567505 -97.87049j
    60.562656 +97.86032j ]
 [  60.576065 +97.865j     -60.58392  -97.86162j  -181.75056 -293.59225j
   -60.55587  -97.856224j]
 [ 181.74327 +293.5806j     60.586044 +97.87458j   -60.5876   -97.87559j
    60.572594 +97.86551j ]]


## 4) Applying 2D IFT (fidelity test)

In [15]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[6.0586487e+01+9.7869766e+01j 6.0586487e+01+9.7869766e+01j
  3.3996105e-03+2.9029846e-03j 6.0586483e+01+9.7869774e+01j]
 [6.0586487e+01+9.7869774e+01j 4.2858124e-03+5.6741238e-03j
  3.0598640e-03+7.4594021e-03j 5.5594444e-03+7.5554848e-04j]
 [7.9100132e-03+8.5000992e-03j 7.3049068e-03+1.4829636e-04j
  9.7513199e-05+5.6180954e-03j 6.2329769e-03+7.7996254e-03j]
 [6.0586483e+01+9.7869774e+01j 6.4988136e-03+6.9434643e-03j
  9.1147423e-03+3.4153461e-03j 6.8483353e-03+5.4905415e-03j]]

Reals MSE: 7.3972365e-12
Imaginaries MSE: 1.12596816e-11


## Utils

### Quantum

### Classical

In [10]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask